In [1]:
import numpy as np
from my_stuff import BinanceUSDMTestKeys, ApexTestKeys, MufexTestKeys, BybitTestKeys,MufexKeys
from quantfreedom.helper_funcs import candles_to_df
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.apex_exchange.apex import Apex
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.strategies.ex_strat_01 import RSIBelowAbove
from datetime import datetime


%load_ext autoreload
%autoreload 2

In [2]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

In [3]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    since_datetime=datetime(2023, 11, 14, 19),
    until_datetime=datetime(2023, 11, 16),
)

candles_to_df(candles=candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2023-11-14 19:00:00,1699988400000,35245.2,35305.7,35109.0,35180.6,30.490
2023-11-14 19:05:00,1699988700000,35180.6,35457.4,35180.6,35419.0,36.090
2023-11-14 19:10:00,1699989000000,35419.0,35453.9,35360.0,35393.5,13.999
2023-11-14 19:15:00,1699989300000,35393.5,35473.4,35360.0,35365.8,14.061
2023-11-14 19:20:00,1699989600000,35365.8,35392.8,35284.8,35333.9,14.049
...,...,...,...,...,...,...
2023-11-15 23:35:00,1700091300000,37822.7,37860.4,37785.4,37860.2,7.898
2023-11-15 23:40:00,1700091600000,37860.2,37896.0,37835.1,37864.7,7.153
2023-11-15 23:45:00,1700091900000,37864.7,37889.9,37804.1,37804.1,5.024


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)
strategy = RSIBelowAbove(
    rsi_is_below=np.array([60, 80]),
    rsi_length=np.array([14]),
    long_short="long",
)

In [5]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=False,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 2
Total order settings to test: 3
Total combinations of settings to test: 6
Total candles: 348
Total candles to test: 2,088


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
0,0,0,10.0,5,5,68.555,50.000,0.690,277.127,685.552,1685.552
1,0,1,7.0,5,2,105.188,71.429,0.899,174.044,1051.876,2051.876
2,0,2,7.0,5,2,92.702,71.429,0.908,163.559,927.015,1927.015
3,1,0,8.0,5,3,75.932,62.500,0.846,207.131,759.317,1759.317
4,1,1,6.0,5,1,90.352,83.333,0.975,129.892,903.518,1903.518
5,1,2,4.0,4,0,73.941,100.000,0.920,74.674,739.408,1739.408


In [8]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=1,
    ind_set_index=1,
    plot_results=True,
    logger_bool=True,
)

In [ ]:
order_records_df

In [ ]:
Mufex(
    api_key=MufexTestKeys.api_key,
    secret_key=MufexTestKeys.secret_key,
    use_test_net=True,
).create_order(
    symbol="BTCUSDT",
    position_mode=PositionModeType.BuySide,
    buy_sell="buy",
    order_type="market",
    asset_size=0.001,
    time_in_force="GoodTillCancel",
)

In [ ]:
Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True,
).create_entry_market_order(
    symbol="BTC-USDC",
    buy_sell="sell",
    asset_size=0.005,
)

In [ ]:
BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
).create_order(
    symbol="BTCUSDT",
    side="BUY",
    type="MARKET",
    quantity=0.005,
    timeInForce=None,
)

In [ ]:
Bybit(
    use_test_net=True,
    api_key=BybitTestKeys.api_key,
    secret_key=BybitTestKeys.secret_key,
).create_order(
    symbol="BTCUSDT",
    buy_sell="buy",
    position_mode=1,
    order_type="market",
    asset_size=0.01,
)